In [14]:
import dotenv
dotenv.load_dotenv('../backend/.env')

import openai
import langchain
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
import numpy as np

In [42]:
COMPARISON_PROMPT = """
You are helping a person writing the current code in the excerpt below. You can suggest them to use one of the snippets below. 
Your goal is to suggest the snippet that does what the current code does in the best way possible.

Current code:
```python
{snippet}
```

Suggested snippet 1: 
```python
{example_1}
```

Suggested snippet 2:
```python
{example_2}
```

Which snippet is the best one? Print only 1 or 2. 
""".strip()

prompt = PromptTemplate(template=COMPARISON_PROMPT, input_variables=["snippet", "example_1", "example_2"])

In [43]:
SNIPPET = '''
from fastapi import FastAPI, File, UploadFile

app = FastAPI()

@app.post("/uploadfile/")
def create_upload_file(file: UploadFile = File(...)):
    """
    Save the uploaded file to the uploads directory
    """

    with open(f"uploads/{file.filename}", "wb") as f:
        f.write(file.file.read())
    
    return {"filename": file.filename, "message": "File uploaded successfully"}
'''

EXAMPLE_1 = """
def upload_image(
        file: UploadFile = File(...),
):
    # 本地存储临时方案，一般生产都是使用第三方云存储OSS(如七牛云, 阿里云)
    # 建议计算并记录一次 文件md5值 避免重复存储相同资源
    save_dir = f"{settings.BASE_PATH}/static/img"

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    try:
        suffix = Path(file.filename).suffix

        with NamedTemporaryFile(delete=False, suffix=suffix, dir=save_dir) as tmp:
            shutil.copyfileobj(file.file, tmp)
            tmp_file_name = Path(tmp.name).name
    finally:
        file.file.close()

    return resp.ok(data={"image": f"/static/img/{tmp_file_name}"})
"""

EXAMPLE_2 = '''
from typing import Union

from fastapi import FastAPI, File, UploadFile
from typing_extensions import Annotated

app = FastAPI()


@app.post("/files/")
async def create_file(file: Annotated[Union[bytes, None], File()] = None):
    if not file:
        return {"message": "No file sent"}
    else:
        return {"file_size": len(file)}


@app.post("/uploadfile/")
async def create_upload_file(file: Union[UploadFile, None] = None):
    if not file:
        return {"message": "No upload file sent"}
    else:
        return {"filename": file.filename}
'''

SNIPPET = SNIPPET.strip()
EXAMPLE_1 = EXAMPLE_1.strip()
EXAMPLE_2 = EXAMPLE_2.strip()

In [44]:
llm = ChatOpenAI(model_name='gpt-4', temperature=0)
chain = prompt | llm

In [45]:
def run_pair_comparison(snippet, example_1, example_2):
    # Swap in 50% of the cases
    swap = np.random.choice([True, False])
    if swap:
        example_1, example_2 = example_2, example_1
    
    # Run the chain
    result = chain.invoke(dict(snippet=snippet, example_1=example_1, example_2=example_2))
    result = result.content
    result = result.strip()
    result = int(result) 

    if swap:
        result = 2 if result == 1 else 1

    result = result - 1
    return result, swap

In [48]:
with get_openai_callback() as cb:
    for i in range(5):
        result = run_pair_comparison(SNIPPET, EXAMPLE_1, EXAMPLE_2)
        print(f'Iteration {i}: {result}')

print(f'Cost of this evaluation {cb.total_cost}$')

Iteration 0: 0
Iteration 1: 0
Iteration 2: 0
Iteration 3: 1
Iteration 4: 0
Cost of this evaluation 0.07830000000000001$


Running comparison with 3 models:

In [53]:
cost_for_3_way_comparison_n_20 = 0.07784999999999999 / 5 * 20 * 3
cost_for_3_way_comparison_n_20

0.9341999999999999